In [1]:
# Train a simple CNN on the MNIST dataset

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import keras
import keras_tqdm
import timeit
import os

Using TensorFlow backend.


In [2]:

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras_tqdm import TQDMNotebookCallback
from keras_tqdm import TQDMCallback
from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot
from keras import optimizers

In [3]:
# training parameters
# batch_size = 128
num_classes = 10
# n_epoch = 20

# for reproducibility
np.random.seed(1234)

In [4]:
from keras.datasets import mnist

In [5]:
# input image dimensions
img_rows, img_cols = 28, 28

# number of convolutional filters to use
n_filters = 32

# size of pooling area for max pooling
pool_size = (2, 2)

# convolution kernel size
kernel_size = (5, 5)

strides = (1, 1)

In [6]:
# shuffle and split the data between training and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [7]:
# set the no of training and test dataset examples

# if Theano backend
if K.image_dim_ordering() == 'th':
    print('Using Theano backend for reshaping the training and test data')
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    # if TensorFlow backend
    print('Using TensorFlow backend for reshaping the training and test data')
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
    
# create placeholders
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# no of training and test batches
x_train /= 255
x_test /= 255
print('x_train_shape: ', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

Using TensorFlow backend for reshaping the training and test data
x_train_shape:  (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [8]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
print(y_train.shape)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_test.shape)

(60000, 10)
(10000, 10)


In [9]:
# Training Parameters
batch_size = 512
n_epochs = 30
learning_rate = 1e-4

In [10]:
# initialize weights
kernel_initializer = 'glorot_uniform'

# initialize biases
bias_initializer = 'zeros'

## Create the CNN Model

In [11]:
# implement the model
model = Sequential()

# First Convolutional Layer
model.add(Conv2D(n_filters, kernel_size=kernel_size, strides=strides, padding='same', 
                 use_bias=True, kernel_initializer=kernel_initializer, bias_initializer=bias_initializer, 
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))

# Second Convolutional Layer
model.add(Conv2D(n_filters, kernel_size=kernel_size, strides=strides, padding='same', use_bias=True, 
                 kernel_initializer=kernel_initializer, bias_initializer=bias_initializer))
model.add(Activation('relu'))

# Apply maxpooling and dropout
model.add(MaxPooling2D(pool_size=pool_size))
# model.add(Dropout(0.25))
 
# Fully Connected Layer
# flatten the output from the second convolutional layer
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
# model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

## Print the Model Architecture

In [12]:
# model summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
activation_2 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              6423552   
__________

In [13]:
# model visualization - using keras utility functions
img = plot_model(model, to_file='mnist_cnn.png')

## Optimize the Model

In [14]:
opt = keras.optimizers.Adam(lr=learning_rate)

# optimize the model
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

## Save Model Checkpoints

In [15]:
# create model checkpoints
save_dir = "checkpoints_mnist_keras_tensorflow/"
# create the directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
# path for the checkpoint file
save_path = os.path.join(save_dir, "mnist_cnn.hdf5")
if os.path.exists(save_path):
    print("Checkpoint created.")
else:
    print("No checkpoint exists as of now!!!!")

checkpointer = ModelCheckpoint(filepath=save_path, verbose=1, save_best_only=True)

No checkpoint exists as of now!!!!


## Helper Class to Record Loss History

In [16]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [17]:
print('Using real-time data augmentation')

start_time = timeit.default_timer()

history = LossHistory()

# perform pre-processing and realtime data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images
        
# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)
        
# Fit the model on the batches generated by datagen.flow().
training = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), verbose=1, 
                               steps_per_epoch=x_train.shape[0] // batch_size, epochs=n_epochs, 
                               callbacks=[TQDMCallback(), checkpointer, history], 
                               validation_data=(x_test, y_test))
        
end_time = timeit.default_timer()
print('Calculating total time for training with data augmentation')
time_diff = end_time - start_time
    
print("The code ran for %.1fs" % (time_diff))
    
print("The code ran for %.2fm" % (time_diff / 60.))

Using real-time data augmentation


Epoch: 0:   0%|          | 0/117 [00:00<?, ?it/s]

Epoch 1/30



Epoch: 0 - loss: 2.313, acc: 0.088  1%|          | 1/117 [00:01<03:12,  1.66s/it]

  1/117 [..............................] - ETA: 192s - loss: 2.3127 - acc: 0.0879

...

115/117 [============================>.] - ETA: 0s - loss: 0.0800 - acc: 0.9752


Epoch: 29 - loss: 0.080, acc: 0.975 99%|█████████▉| 116/117 [00:16<00:00,  6.97it/s]

116/117 [============================>.] - ETA: 0s - loss: 0.0799 - acc: 0.9752


Epoch: 29 - loss: 0.080, acc: 0.975, val_loss: 0.055, val_acc: 0.983100%|██████████| 117/117 [00:17<00:00,  2.91it/s]
Training: 100%|██████████| 30/30 [08:42<00:00, 17.29s/it]

Epoch 00029: val_loss improved from 0.05950 to 0.05481, saving model to checkpoints_mnist_keras_tensorflow/mnist_cnn.hdf5
117/117 [==============================] - 17s - loss: 0.0796 - acc: 0.9753 - val_loss: 0.0548 - val_acc: 0.9827
Calculating total time for training with data augmentation
The code ran for 523.5s
The code ran for 8.72m


In [18]:
# evaluate the model
# find the test score and accuracy
score = model.evaluate(x_test, y_test, verbose=0)

print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Test Loss: 0.0548133831302
Test accuracy: 0.9827
